In [ ]:
pip install nearpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 3.3 MB/s 
     |████████████████████████████████| 241 kB 65.1 MB/s 


In [ ]:
print(__doc__)
import matplotlib
#matplotlib.use('TkAgg')

import heapq
import numpy as np
import pandas as pd
import scipy as sp
import math
from scipy import spatial
import matplotlib.pyplot as plt
import tensorflow as tf
import copy

class FacilityLocation:

    def __init__(self, D, V, alpha=1.):
        '''
        Args
        - D: np.array, shape [N, N], similarity matrix
        - V: list of int, indices of columns of D
        - alpha: float
        '''
        self.D = D
        self.curVal = 0
        self.curMax = np.zeros(len(D))
        self.gains = []
        self.alpha = alpha
        self.f_norm = self.alpha / self.f_norm(V)
        self.norm = 1. / self.inc(V, [])

    def f_norm(self, sset):
        return self.D[:, sset].max(axis=1).sum()

    def inc(self, sset, ndx):
        if len(sset + [ndx]) > 1:
            if not ndx:  # normalization
                return math.log(1 + self.alpha * 1)
            return self.norm * math.log(1 + self.f_norm * np.maximum(self.curMax, self.D[:, ndx]).sum()) - self.curVal
        else:
            return self.norm * math.log(1 + self.f_norm * self.D[:, ndx].sum()) - self.curVal

    def add(self, sset, ndx):
        cur_old = self.curVal
        if len(sset + [ndx]) > 1:
            self.curMax = np.maximum(self.curMax, self.D[:, ndx])
        else:
            self.curMax = self.D[:, ndx]
        self.curVal = self.norm * math.log(1 + self.f_norm * self.curMax.sum())
        self.gains.extend([self.curVal - cur_old])
        return self.curVal


def _heappush_max(heap, item):
    heap.append(item)
    heapq._siftdown_max(heap, 0, len(heap)-1)


def _heappop_max(heap):
    """Maxheap version of a heappop."""
    lastelt = heap.pop()  # raises appropriate IndexError if heap is empty
    if heap:
        returnitem = heap[0]
        heap[0] = lastelt
        heapq._siftup_max(heap, 0)
        return returnitem
    return lastelt


def lazy_greedy_heap(F, V, B):
    curVal = 0
    sset = []
    vals = []

    order = []
    heapq._heapify_max(order)
    [_heappush_max(order, (F.inc(sset, index), index)) for index in V]

    while order and len(sset) < B:
        el = _heappop_max(order)
        improv = F.inc(sset, el[1])

        # check for uniques elements
        if improv >= 0:
            if not order:
                curVal = F.add(sset, el[1])
                sset.append(el[1])
                vals.append(curVal)
            else:
                top = _heappop_max(order)
                if improv >= top[0]:
                    curVal = F.add(sset, el[1])
                    sset.append(el[1])
                    vals.append(curVal)
                else:
                    _heappush_max(order, (improv, el[1]))
                _heappush_max(order, top)

    return sset, vals


def test():
    n = 10
    X = np.random.rand(n, n)
    D = X * np.transpose(X)
    F = FacilityLocation(D, range(0, n))
    sset = lazy_greedy(F, range(0, n), 15)
    print(sset)



Automatically created module for IPython interactive environment


## Implementing Necessary Functions for CRAIG

In [ ]:
import itertools
import os
import subprocess
import time
import gc

from nearpy import Engine
from nearpy.distances import EuclideanDistance
from nearpy.filters import NearestFilter
from nearpy.hashes import RandomBinaryProjections
import matplotlib.pyplot as plt
import numpy as np
#from lazy_greedy import FacilityLocation, lazy_greedy_heap
import scipy.spatial
# from eucl_dist.cpu_dist import dist
# from eucl_dist.gpu_dist import dist as gdist
#import tensorflow.compat.v2 as tf
#tf.disable_v2_behavior()
import tensorflow
from multiprocessing.dummy import Pool as ThreadPool
from itertools import repeat
import sklearn
# from lazy_greedy import FacilityLocation, lazy_greedy, lazy_greedy_heap
# from set_cover import SetCover
import keras
from keras.datasets import mnist

SEED = 100
EPS = 1E-8
PLOT_NAMES = ['lr', 'data_loss', 'epoch_loss', 'test_loss']  # 'cluster_compare', 'cosine_compare', 'euclidean_compare'


def load_dataset(dataset, dataset_dir):
    '''
    Args
    - dataset: str, one of ['cifar10', 'covtype'] or filename in `data/`
    - dataset_dir: str, path to `data` folder

    Returns
    - X: np.array, shape [N, d]
      - exception: shape [N, 32, 32, 3] for cifar10
    - y: np.array, shape [N]
    '''
    if dataset == 'cifar10':
        path = os.path.join(dataset_dir, 'cifar10', 'cifar10.npz')
        with np.load(path) as npz:
            X = npz['x']  # shape [60000, 32, 32, 3], type uint8
            y = npz['y']  # shape [60000], type uint8
        # convert to float in (0, 1), center at mean 0
        X = X.astype(np.float32) / 255
        # X -= np.mean(X, axis=0)
    elif dataset == 'cifar10_features':
        path = os.path.join(dataset_dir, 'cifar10', 'train_features.npz')
        with np.load(path) as npz:
            X = npz['features']  # shape [50000, 64], type float32
            y = npz['labels']  # shape [50000], type int64
    elif dataset == 'cifar10_grads':
        # labels
        path = os.path.join(dataset_dir, 'cifar10', 'train_features.npz')
        with np.load(path) as npz:
            y = npz['labels']  # shape [50000], type int64
        # feautres
        path = os.path.join('grad_features.npy')
        X = np.load(path)  # shape [50000, 1000], type float16
    elif dataset == 'mnist':
        X_train = np.vstack([mnist.train.images, mnist.validation.images])
        y_train = np.hstack([mnist.train.labels, mnist.validation.labels])
        X_test = mnist.test.images
        y_test = mnist.test.labels
        X_train = X_train.astype(np.float32) / 255
        X_test = X_test.astype(np.float32) / 255
        return X_train, y_train, X_test, y_test

    else:
        num, dim, name = 0, 0, ''
        if dataset == 'covtype':
            num, dim = 581012, 54
            name = 'covtype.libsvm.binary.scale'
        elif dataset == 'ijcnn1.t' or dataset == 'ijcnn1.tr':
            num, dim = 49990 if 'tr' in dataset else 91701, 22
            name = dataset
        elif dataset == 'combined_scale' or dataset == 'combined_scale.t':
            num, dim = 19705 if '.t' in dataset else 78823, 100
            name = dataset

        X = np.zeros((num, dim), dtype=np.float32)
        y = np.zeros(num, dtype=np.int32)
        path = os.path.join(dataset_dir, name)

        with open(path, 'r') as f:
            for i, line in enumerate(f):
                y[i] = float(line.split()[0])
                for e in line.split()[1:]:
                    cur = e.split(':')
                    X[i][int(cur[0]) - 1] = float(cur[1])
                i += 1
        y = np.array(y, dtype=np.int32)
        if name in ['ijcnn1.t', 'ijcnn1.tr']:
            y[y == -1] = 0
        else:
            y = y - np.ones(len(y), dtype=np.int32)

    return X, y


def similarity(X, metric):
    '''Computes the similarity between each pair of examples in X.

    Args
    - X: np.array, shape [N, d]
    - metric: str, one of ['cosine', 'euclidean']

    Returns
    - S: np.array, shape [N, N]
    '''
    # print(f'Computing similarity for {metric}...', flush=True)
    start = time.time()
    dists = sklearn.metrics.pairwise_distances(X, metric=metric, n_jobs=1)
    # dists = gdist(X, X, optimize_level=0, output='cpu')
    elapsed = time.time() - start

    if metric == 'cosine':
        S = 1 - dists
    elif metric == 'euclidean' or metric == 'l1':
        m = np.max(dists)
        S = m - dists
    else:
        raise ValueError(f'unknown metric: {metric}')

    return S, elapsed


def greedy_merge(X, y, B, part_num, metric, smtk=0, stoch_greedy=False):
    N = len(X)
    indices = list(range(N))
    # np.random.shuffle(indices)
    part_size = int(np.ceil(N / part_num))
    part_indices = [indices[slice(i * part_size, min((i + 1) * part_size, N))] for i in range(part_num)]
    print(f'GreeDi with {part_num} parts, finding {B} elements...', flush=True)

    # pool = ThreadPool(part_num)
    # order_mg_all, cluster_sizes_all, _, _, ordering_time, similarity_time = zip(*pool.map(
    #     lambda p: get_orders_and_weights(
    #         int(B / 2), X[part_indices[p], :], metric, p + 1, stoch_greedy, y[part_indices[p]]), np.arange(part_num)))
    # pool.terminate()

    order_mg_all, cluster_sizes_all, _, _, ordering_time, similarity_time, F_val = zip(*map(
        lambda p: get_orders_and_weights(
            int(B / 2), X[part_indices[p], :], metric, p + 1, stoch_greedy, y[part_indices[p]]), np.arange(part_num)))

    # Returns the number of objects it has collected and deallocated
    collected = gc.collect()
    print(f'Garbage collector: collected {collected}')

    # order_mg_all = np.zeros((part_num, B))
    # cluster_sizes_all = np.zeros((part_num, B))
    # ordering_time = np.zeros(part_num)
    # similarity_time = np.zeros(part_num)
    # for p in range(part_num):
    #    order_mg_all[p,:], cluster_sizes_all[p,:], _, _, ordering_time[p], similarity_time[p] = get_orders_and_weights(
    #         B, X[part_indices[p], :], metric, p, stoch_greedy, y[part_indices[p]])
    order_mg_all = np.array(order_mg_all, dtype=np.int32)
    cluster_sizes_all = np.array(cluster_sizes_all, dtype=np.float32)  # / part_num (not needed)
    order_mg = order_mg_all.flatten(order='F')
    weights_mg = cluster_sizes_all.flatten(order='F')
    print(f'GreeDi stage 1: found {len(order_mg)} elements in: {np.max(ordering_time)} sec', flush=True)

    # order_mg, weights_mg, order_sz, weights_sz, ordering_time, similarity_time
    order, weights, order_sz, weights_sz, ordering_time_merge, similarity_time_merge = get_orders_and_weights(
        B, X[order_mg, :], metric, smtk, 0, stoch_greedy, y[order_mg], weights_mg)
    print(weights)
    total_ordering_time = np.max(ordering_time) + ordering_time_merge
    total_similarity_time = np.max(similarity_time) + similarity_time_merge
    print(f'GreeDi stage 2: found {len(order)} elements in: {total_ordering_time + total_similarity_time} sec',
          flush=True)
    vals = order, weights, order_sz, weights_sz, total_ordering_time, total_similarity_time
    return vals


def greedi(X, y, B, part_num, metric, smtk=0, stoch_greedy=False, seed=-1):
    N = len(X)
    indices = list(range(N))
    if seed != -1:
        np.random.seed(seed)
        np.random.shuffle(indices)  # Note: random shuffling
    part_size = int(np.ceil(N / part_num))
    part_indices = [indices[slice(i * part_size, min((i + 1) * part_size, N))] for i in range(part_num)]
    print(f'GreeDi with {part_num} parts, finding {B} elements...', flush=True)

    # pool = ThreadPool(part_num)
    # order_mg_all, cluster_sizes_all, _, _, ordering_time, similarity_time = zip(*pool.map(
    #     lambda p: get_orders_and_weights(
    #         B, X[part_indices[p], :], metric, p + 1, stoch_greedy, y[part_indices[p]]), np.arange(part_num)))
    # pool.terminate()
    # Returns the number of objects it has collected and deallocated
    # collected = gc.collect()
    # print(f'Garbage collector: collected {collected}')
    order_mg_all, cluster_sizes_all, _, _, ordering_time, similarity_time = zip(*map(
        lambda p: get_orders_and_weights(
            B, X[part_indices[p], :], metric, p + 1, stoch_greedy, y[part_indices[p]]), np.arange(part_num)))
    gc.collect()

    order_mg_all = np.array(order_mg_all, dtype=np.int32)
    for c in np.arange(part_num):
        order_mg_all[c] = np.array(part_indices[c])[order_mg_all[c]]
    # order_mg_all = np.zeros((part_num, B))
    # cluster_sizes_all = np.zeros((part_num, B))
    # ordering_time = np.zeros(part_num)
    # similarity_time = np.zeros(part_num)
    # for p in range(part_num):
    #    order_mg_all[p,:], cluster_sizes_all[p,:], _, _, ordering_time[p], similarity_time[p] = get_orders_and_weights(
    #         B, X[part_indices[p], :], metric, p, stoch_greedy, y[part_indices[p]])
    cluster_sizes_all = np.array(cluster_sizes_all, dtype=np.float32)  # / part_num (not needed)
    order_mg = order_mg_all.flatten(order='F')
    weights_mg = cluster_sizes_all.flatten(order='F')
    print(f'GreeDi stage 1: found {len(order_mg)} elements in: {np.max(ordering_time)} sec', flush=True)

    # order_mg, weights_mg, order_sz, weights_sz, ordering_time, similarity_time
    order, weights, order_sz, weights_sz, ordering_time_merge, similarity_time_merge = get_orders_and_weights(
        B, X[order_mg,:], metric, smtk, 0, stoch_greedy, y[order_mg], weights_mg)
    print(weights)
    order = order_mg[order]
    total_ordering_time = np.max(ordering_time) + ordering_time_merge
    total_similarity_time = np.max(similarity_time) + similarity_time_merge
    print(f'GreeDi stage 2: found {len(order)} elements in: {total_ordering_time + total_similarity_time} sec', flush=True)
    vals = order, weights, order_sz, weights_sz, total_ordering_time, total_similarity_time
    return vals


def get_facility_location_submodular_order(S, B, c, smtk=0, no=0, stoch_greedy=0, weights=None):
    '''
    Args
    - S: np.array, shape [N, N], similarity matrix
    - B: int, number of points to select

    Returns
    - order: np.array, shape [B], order of points selected by facility location
    - sz: np.array, shape [B], type int64, size of cluster associated with each selected point
    '''
    # print('Computing facility location submodular order...')
    N = S.shape[0]
    no = smtk if no == 0 else no

    if smtk > 0:
        print(f'Calculating ordering with SMTK... part size: {len(S)}, B: {B}', flush=True)
        np.save(f'/tmp/{no}/{smtk}-{c}', S)
        if stoch_greedy > 0:
            p = subprocess.check_output(
                f'/tmp/{no}/smtk-master{smtk}/build/smraiz -sumsize {B} \
                 -stochastic-greedy -sg-epsilon {stoch_greedy} -flnpy /tmp/{no}/{smtk}-{c}.'
                f'npy -pnpv -porder -ptime'.split())
        else:
            p = subprocess.check_output(
                f'/tmp/{no}/smtk-master{smtk}/build/smraiz -sumsize {B} \
                             -flnpy /tmp/{no}/{smtk}-{c}.npy -pnpv -porder -ptime'.split())
        s = p.decode("utf-8")
        str, end = ['([', ',])']
        order = s[s.find(str) + len(str):s.rfind(end)].split(',')
        greedy_time = float(s[s.find('CPU') + 4 : s.find('s (User')])
        str = 'f(Solution) = '
        F_val = float(s[s.find(str) + len(str) : s.find('Summary Solution') - 1])
    else:
        V = list(range(N))
        start = time.time()
        F = FacilityLocation(S, V)
        order, _ = lazy_greedy_heap(F, V, B)
        greedy_time = time.time() - start
        F_val = 0

    order = np.asarray(order, dtype=np.int64)
    sz = np.zeros(B, dtype=np.float64)
    for i in range(N):
        if weights is None:
            sz[np.argmax(S[i, order])] += 1
        else:
            sz[np.argmax(S[i, order])] += weights[i]
    # print('time (sec) for computing facility location:', greedy_time, flush=True)
    collected = gc.collect()
    return order, sz, greedy_time, F_val


def faciliy_location_order(c, X, y, metric, num_per_class, smtk, no, stoch_greedy, weights=None):
    class_indices = np.where(y == c)[0]
    print(c)
    print(class_indices)
    print(len(class_indices))
    S, S_time = similarity(X[class_indices], metric=metric)
    order, cluster_sz, greedy_time, F_val = get_facility_location_submodular_order(
        S, num_per_class, c, smtk, no, stoch_greedy, weights)
    return class_indices[order], cluster_sz, greedy_time, S_time


def save_all_orders_and_weights(folder, X, metric='l2', stoch_greedy=False, y=None, equal_num=False, outdir='.'):
    N = X.shape[0]
    if y is None:
        y = np.zeros(N, dtype=np.int32)  # assign every point to the same class
    classes = np.unique(y)
    C = len(classes)  # number of classes
    # assert np.array_equal(classes, np.arange(C))
    # assert B % C == 0
    class_nums = [sum(y == c) for c in classes]
    print(class_nums)
    class_indices = [np.where(y == c)[0] for c in classes]

    tmp_path = '/tmp'
    no, smtk = 2, 2

    def greedy(B, c):
        print('Computing facility location submodular order...')
        print(f'Calculating ordering with SMTK... part size: {class_nums[c]}, B: {B}', flush=True)
        command = f'/tmp/{no}/smtk-master{smtk}/build/smraiz -sumsize {B} \
                                 -flnpy {tmp_path}/{no}/{smtk}-{c}.npy -pnpv -porder -ptime'
        if stoch_greedy:
            command += f' -stochastic-greedy -sg-epsilon {.9}'

        p = subprocess.check_output(command.split())
        s = p.decode("utf-8")
        str, end = ['([', ',])']
        order = s[s.find(str) + len(str):s.rfind(end)].split(',')
        order = np.asarray(order, dtype=np.int64)
        greedy_time = float(s[s.find('CPU') + 4: s.find('s (User')])
        print(f'FL greedy time: {greedy_time}', flush=True)
        str = 'f(Solution) = '
        F_val = float(s[s.find(str) + len(str) : s.find('Summary Solution') - 1])
        print(f'===========> f(Solution) = {F_val}')
        print('time (sec) for computing facility location:', greedy_time, flush=True)
        return order, greedy_time, F_val

    def get_subset_sizes(B, equal_num):
        if equal_num:
            # class_nums = [sum(y == c) for c in classes]
            num_per_class = int(np.ceil(B / C)) * np.ones(len(classes), dtype=np.int32)
            minority = class_nums < np.ceil(B / C)
            if sum(minority) > 0:
                extra = sum([max(0, np.ceil(B / C) - class_nums[c]) for c in classes])
                for c in classes[~minority]:
                    num_per_class[c] += int(np.ceil(extra / sum(minority)))
        else:
            num_per_class = np.int32(np.ceil(np.divide([sum(y == i) for i in classes], N) * B))

        return num_per_class

    def merge_orders(order_mg_all, weights_mg_all, equal_num):
        order_mg, weights_mg = [], []
        if equal_num:
            props = np.rint([len(order_mg_all[i]) for i in range(len(order_mg_all))])
        else:
            # merging imbalanced classes
            class_ratios = np.divide([np.sum(y == i) for i in classes], N)
            props = np.rint(class_ratios / np.min(class_ratios))
            print(f'Selecting with ratios {np.array(class_ratios)}')
            print(f'Class proportions {np.array(props)}')

        order_mg_all = np.array(order_mg_all)
        weights_mg_all = np.array(weights_mg_all)
        for i in range(int(np.rint(np.max([len(order_mg_all[c]) / props[c] for c in classes])))):
            for c in classes:
                ndx = slice(i * int(props[c]), int(min(len(order_mg_all[c]), (i + 1) * props[c])))
                order_mg = np.append(order_mg, order_mg_all[c][ndx])
                weights_mg = np.append(weights_mg, weights_mg_all[c][ndx])
        order_mg = np.array(order_mg, dtype=np.int32)
        weights_mg = np.array(weights_mg, dtype=np.float)
        return order_mg, weights_mg

    def calculate_weights(order, c):
        weight = np.zeros(len(order), dtype=np.float64)
        center = np.argmax(D[str(c)][:, order], axis=1)
        for i in range(len(order)):
            weight[i] = np.sum(center == i)
        return weight

    D, m = {}, 0
    similarity_times, max_similarity = [], []
    for c in classes:
        print(f'Computing distances for class {c}...')
        time.sleep(.1)
        start = time.time()
        if metric in ['', 'l2', 'l1']:
            dists = sklearn.metrics.pairwise_distances(X[class_indices[c]], metric=metric, n_jobs=1)
        else:
            p = float(metric)
            dim = class_nums[c]
            dists = np.zeros((dim, dim))
            for i in range(dim):
                dists[i,:] = np.power(np.sum(np.power(np.abs(X[class_indices[c][i]] - X[class_indices[c]]), p), axis=1), 1./p)
                # for j in range(i+1, dim):
                #     dists[i,j] = np.power(np.sum(np.power(np.abs(X[class_indices[c][i]] - X[class_indices[c][j]]), p)), 1./p)
            # dists[np.triu_indices(dim, 1)] = d
            # dists = dists.T + dists
        similarity_times.append(time.time() - start)
        print(f'similarity times: {similarity_times}')
        print('Computing max')
        m = np.max(dists)
        print(f'max: {m}')
        S = m - dists
        np.save(f'{tmp_path}/{no}/{smtk}-{c}', S)
        D[str(c)] = S
        max_similarity.append(m)

    # Ordering all the data with greedy
    print(f'Greedy: selecting {class_nums} elements')
    # order_in_class, greedy_times, F_vals = zip(*map(lambda c: greedy(class_nums[c], c), classes))
    # order_all = [class_indices[c][order_in_class[c]] for c in classes]

    for subset_size in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    # for subset_size in [0.9, 1.0]:
        B = int(N * subset_size)
        num_per_class = get_subset_sizes(B, equal_num)

        # Note: for marginal gains
        order_in_class, greedy_times, F_vals = zip(*map(lambda c: greedy(num_per_class[c], c), classes))
        order_all = [class_indices[c][order_in_class[c]] for c in classes]
        #####

        weights = [calculate_weights(order_in_class[c][:num_per_class[c]], c) for c in classes]
        order_subset = [order_all[c][:num_per_class[c]] for c in classes]
        order_merge, weights_merge = merge_orders(order_subset, weights, equal_num)
        F_vals = np.divide(F_vals, class_nums)

        folder = '/tmp/covtype'
        print(f'saving to {folder}_{subset_size}_{metric}_w.npz')
        np.savez(f'{folder}_{subset_size}_{metric}_w', order=order_merge, weight=weights_merge,
                 order_time=greedy_times, similarity_time=similarity_times, F_vals=F_vals, max_dist=m)
    # end for on subset sizes
    # return vals


def get_orders_and_weights(B, X, metric, smtk, no=0, stoch_greedy=0, y=None, weights=None, equal_num=False, outdir='.'):
    '''
    Ags
    - X: np.array, shape [N, d]
    - B: int, number of points to select
    - metric: str, one of ['cosine', 'euclidean'], for similarity
    - y: np.array, shape [N], integer class labels for C classes
      - if given, chooses B / C points per class, B must be divisible by C
    - outdir: str, path to output directory, must already exist

    Returns
    - order_mg/_sz: np.array, shape [B], type int64
      - *_mg: order points by their marginal gain in FL objective (largest gain first)
      - *_sz: order points by their cluster size (largest size first)
    - weights_mg/_sz: np.array, shape [B], type float32, sums to 1
    '''
    N = X.shape[0]
    if y is None:
        y = np.zeros(N, dtype=np.int32)  # assign every point to the same class
    classes = np.unique(y)
    C = len(classes)  # number of classes
    # assert np.array_equal(classes, np.arange(C))
    # assert B % C == 0

    if equal_num:
        class_nums = [sum(y == c) for c in classes]
        num_per_class = int(np.ceil(B / C)) * np.ones(len(classes), dtype=np.int32)
        minority = class_nums < np.ceil(B / C)
        if sum(minority) > 0:
            extra = sum([max(0, np.ceil(B / C) - class_nums[c]) for c in classes])
            for c in classes[~minority]:
                num_per_class[c] += int(np.ceil(extra / sum(minority)))
    else:
        num_per_class = np.int32(np.ceil(np.divide([sum(y == i) for i in classes], N) * B))
        print('not equal_num')

    # print(f'Greedy: selecting {num_per_class} elements')

    # order_mg_all = np.zeros([C, num_per_class], dtype=np.int64)
    # cluster_sizes_all = np.zeros([C, num_per_class], dtype=np.float32)
    # greedy_time_all = np.zeros([C, num_per_class], dtype=np.int64)
    # similarity_time_all = np.zeros([C, num_per_class], dtype=np.int64)

    # pool = ThreadPool(C)
    # order_mg_all, cluster_sizes_all, greedy_times, similarity_times = zip(*pool.map(
    #     lambda c: faciliy_location_order(c, X, y, metric, num_per_class[c], smtk, stoch_greedy, weights), classes))
    # pool.terminate()
    order_mg_all, cluster_sizes_all, greedy_times, similarity_times = zip(*map(
        lambda c: faciliy_location_order(c, X, y, metric, num_per_class[c], smtk, no, stoch_greedy, weights), classes))

    order_mg, weights_mg = [], []
    if equal_num:
        props = np.rint([len(order_mg_all[i]) for i in range(len(order_mg_all))])
    else:
        # merging imbalanced classes
        class_ratios = np.divide([np.sum(y == i) for i in classes], N)
        props = np.rint(class_ratios / np.min(class_ratios))
        print(f'Selecting with ratios {np.array(class_ratios)}')
        print(f'Class proportions {np.array(props)}')

    order_mg_all = np.array(order_mg_all)
    cluster_sizes_all = np.array(cluster_sizes_all)
    for i in range(int(np.rint(np.max([len(order_mg_all[c]) / props[c] for c in classes])))):
        for c in classes:
            ndx = slice(i * int(props[c]), int(min(len(order_mg_all[c]), (i + 1) * props[c])))
            order_mg = np.append(order_mg, order_mg_all[c][ndx])
            weights_mg = np.append(weights_mg, cluster_sizes_all[c][ndx])
    order_mg = np.array(order_mg, dtype=np.int32)

    # class_ratios = np.divide([np.sum(y == i) for i in classes], N)
    # weights_mg[y[order_mg] == np.argmax(class_ratios)] /= (np.max(class_ratios) / np.min(class_ratios))

    weights_mg = np.array(weights_mg, dtype=np.float32)
    ordering_time = np.max(greedy_times)
    similarity_time = np.max(similarity_times)

    # for c in classes:
    #     class_indices = np.where(y == c)[0]
    #     S, similarity_time_all[c] = similarity(X[class_indices], metric=metric)
    #     order, cluster_sz, greedy_time_all[c], F_val = get_facility_location_submodular_order(S, num_per_class, c, smtk)
    #     order_mg_all[c] = class_indices[order]
    #     cluster_sizes_all[c] = cluster_sz
    #     save_cluster_sizes(cluster_sizes_all[c], metric=f'{metric}_class{c}', outdir=outdir)
    # cluster_sizes_all /= N

    # choose 1st from each class, then 2nd from each class, etc.
    # i.e. column-major order
    # order_mg_all = np.array(order_mg_all)
    # cluster_sizes_all = np.array(cluster_sizes_all, dtype=np.float32) / N
    # order_mg = order_mg_all.flatten(order='F')
    # weights_mg = cluster_sizes_all.flatten(order='F')

    # sort by descending cluster size within each class
    # cluster_order = np.argsort(-cluster_sizes_all, axis=1)
    # rows_selector = np.arange(C)[:, np.newaxis]
    order_sz = []  # order_mg_all[rows_selector, cluster_order].flatten(order='F')
    weights_sz = [] # cluster_sizes_all[rows_selector, cluster_order].flatten(order='F')
    vals = order_mg, weights_mg, order_sz, weights_sz, ordering_time, similarity_time
    return vals

## Computing Forgetting Statistics

In [ ]:
def compute_forgetting_statistics(scores, percentage = 5):

    # Forgetting event is a transition in accuracy from 1 to 0
    fgt_events = scores[:, 1:] - scores[:, :-1]

    # Count of forgetting events for each point
    fgt_ncounts = np.sum(fgt_events==-1, axis=1)

    # Indices of points to be removed based on percentage
    fgt_indices = np.argsort(fgt_ncounts)[:int(np.floor(scores.shape[0] * (float(percentage)/100)))]
    
    return fgt_indices

## Running F-CRAIG

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
import numpy as np
import time
from keras.regularizers import l2
import sklearn.metrics
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

total_start = time.time()
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)

num_classes, smtk = 10, 0
Y_train_nocat = Y_train
Y_train = np_utils.to_categorical(Y_train, num_classes)
Y_test = np_utils.to_categorical(Y_test, num_classes)

batch_size = 32
# subset, random = False, False  # all
subset, random = True, False  # greedy
# subset, random = True, True  # random
subset_size = .4 if subset else 1.0
epochs = 10
reg = 1e-4
runs = 3
save_subset = False
use_forgettability_score = True

folder = f'/content/'


model = Sequential()
model.add(Dense(100, input_dim=784, kernel_regularizer=l2(reg)))
model.add(Activation('sigmoid'))
model.add(Dense(10, kernel_regularizer=l2(reg)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='sgd')


train_loss, test_loss = np.zeros((runs, epochs)), np.zeros((runs, epochs))
train_acc, test_acc = np.zeros((runs, epochs)), np.zeros((runs, epochs))
train_time = np.zeros((runs, epochs))
fgt_score_time = np.zeros((runs, epochs))
epoch_time, craig_time = np.zeros((runs, epochs)), np.zeros((runs, epochs))
grd_time, sim_time, pred_time, gow_time = np.zeros((runs, epochs)), np.zeros((runs, epochs)), np.zeros((runs, epochs)), np.zeros((runs, epochs))
not_selected = np.zeros((runs, epochs))
times_selected = np.zeros((runs, len(X_train)))
best_acc = 0
print(f'----------- smtk: {smtk} ------------')

if save_subset:
    B = int(subset_size * len(X_train))
    selected_ndx = np.zeros((runs, epochs, B))
    selected_wgt = np.zeros((runs, epochs, B))

for run in range(runs):
    X_subset = X_train
    Y_subset = Y_train
    W_subset = np.ones(len(X_subset))
    ordering_time,similarity_time, pre_time = 0, 0, 0
    loss_vec, acc_vec, time_vec = [], [], []
    
    fgt_scores = np.empty([len(X_train), 0])
    keep_indices = list(range(len(X_train)))
    
    for epoch in range(0, epochs):
        start_e = time.time()
        print('Epoch {}/{}'.format(epoch, epochs - 1))
        num_batches = int(np.ceil(X_subset.shape[0] / float(batch_size)))

        for index in range(num_batches):
            X_batch = X_subset[index * batch_size:(index + 1) * batch_size]
            Y_batch = Y_subset[index * batch_size:(index + 1) * batch_size]
            W_batch = W_subset[index * batch_size:(index + 1) * batch_size]

            start = time.time()
            history = model.train_on_batch(X_batch, Y_batch, sample_weight=W_batch)
            train_time[run][epoch] += time.time() - start

            start = time.time()
            if (index % 100)==0 and use_forgettability_score:
                Y_pred = model.predict(X_train[keep_indices])
                Y_pred_class = np.argmax(Y_pred, axis=1)
                batch_acc = (Y_train_nocat[keep_indices]==Y_pred_class).astype(int)
                fgt_scores = np.concatenate([fgt_scores, np.expand_dims(batch_acc, axis=1)], axis=1)
            fgt_score_time[run][epoch]+=time.time() - start
        
        if use_forgettability_score and epoch>0:

            start = time.time()
            fgt_indices = compute_forgetting_statistics(fgt_scores, percentage=5)
            keep_indices = [keep_indices[i] for i in range(len(keep_indices)) if i not in fgt_indices]

            # Update forgetting scores
            fgt_scores = fgt_scores[[i for i in range(len(fgt_scores)) if i not in fgt_indices]]
            fgt_score_time[run][epoch]+=time.time() - start

        start_c = time.time()
        if subset:
            if random:
                # indices = np.random.randint(0, len(X_train), int(subset_size * len(X_train)))
                indices = copy.deepcopy(keep_indices)
                np.random.shuffle(indices)
                indices = indices[:int(subset_size * len(X_train))]
                W_subset = np.ones(len(indices))
            else:
                start = time.time()
                _logits = model.predict(X_train[keep_indices])
                pre_time = time.time() - start
                features = _logits - Y_train[keep_indices]
                start = time.time()
                indices, W_subset, _, _, ordering_time, similarity_time = get_orders_and_weights(
                    int(subset_size * len(X_train)), features, 'euclidean', smtk, 0, False, Y_train_nocat[keep_indices])
                indices = [keep_indices[i] for i in indices]
                gow_time[run, epoch] = time.time() - start

                W_subset = W_subset / np.sum(W_subset) * len(W_subset)  # todo

            if save_subset:
                selected_ndx[run, epoch], selected_wgt[run, epoch] = indices, W_subset

            grd_time[run, epoch], sim_time[run, epoch], pred_time[run, epoch]  = ordering_time, similarity_time, pre_time
            times_selected[run][indices] += 1
            not_selected[run, epoch] = np.sum(times_selected[run] == 0) / len(times_selected[run]) * 100
        else:
            pred_time = 0
            indices = np.arange(len(X_train))

        craig_time[run][epoch] = time.time() - start_c
        X_subset = X_train[indices, :]
        Y_subset = Y_train[indices]

        start = time.time()
        score = model.evaluate(X_test, Y_test, verbose=1)
        eval_time = time.time()-start

        start = time.time()
        score_loss = model.evaluate(X_train[keep_indices], Y_train[keep_indices], verbose=1)
        print(f'eval time on training: {time.time()-start}')

        test_loss[run][epoch], test_acc[run][epoch] = score[0], score[1]
        train_loss[run][epoch], train_acc[run][epoch] = score_loss[0], score_loss[1]
        best_acc = max(test_acc[run][epoch], best_acc)

        grd = 'random_wor' if random else 'grd_normw'
        print(f'run: {run}, {grd}, subset_size: {subset_size}, epoch: {epoch}, test_acc: {test_acc[run][epoch]}, '
              f'loss: {train_loss[run][epoch]}, best_prec1_gb: {best_acc}, not selected %:{not_selected[run][epoch]}')
        
        epoch_time[run][epoch] = time.time() - start_e
    if save_subset:
        print(
            f'Saving the results to {folder}_{subset_size}_{grd}_{runs}_fcraig')

        np.savez(f'{folder}_{subset_size}_{grd}_{runs}_fcraig',
                 # f'_{grd}_{args.lr_schedule}_start_{args.start_subset}_lag_{args.lag}_subset',
                 train_loss=train_loss, test_acc=test_acc, train_acc=train_acc, test_loss=test_loss,
                 train_time=train_time, grd_time=grd_time, sim_time=sim_time, pred_time=pred_time,
                 not_selected=not_selected, times_selected=times_selected,
                 subset=selected_ndx, weights=selected_wgt,
                 fgt_score_time=fgt_score_time, gow_time = gow_time,
                 epoch_time=epoch_time, craig_time=craig_time)
    else:
        print(
            f'Saving the results to {folder}_{subset_size}_{grd}_{runs}_fcraig')

        np.savez(f'{folder}_{subset_size}_{grd}_{runs}_fcraig',
                 # f'_{grd}_{args.lr_schedule}_start_{args.start_subset}_lag_{args.lag}',
                 train_loss=train_loss, test_acc=test_acc, train_acc=train_acc, test_loss=test_loss,
                 train_time=train_time, grd_time=grd_time, sim_time=sim_time, pred_time=pred_time,
                 not_selected=not_selected, times_selected=times_selected,
                 fgt_score_time=fgt_score_time, gow_time = gow_time,
                 epoch_time=epoch_time, craig_time=craig_time)
        
total_time = time.time() - total_start
print(total_time)

Instructions for updating:
non-resource variables are not supported in the long term


11490434/11490434 [==============================] - 1s 0us/step
----------- smtk: 0 ------------
Epoch 0/9


/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


not equal_num
0
[    1    21    34 ... 59952 59972 59987]
5923
1
[    3     6     8 ... 59979 59984 59994]
6742
2
[    5    16    25 ... 59983 59985 59991]
5958
3
[    7    10    12 ... 59978 59980 59996]
6131
4
[    2     9    20 ... 59943 59951 59975]
5842
5
[    0    11    35 ... 59968 59993 59997]
5421
6
[   13    18    32 ... 59982 59986 59998]
5918
7
[   15    29    38 ... 59963 59977 59988]
6265
8
[   17    31    41 ... 59989 59995 59999]
5851
9
[    4    19    22 ... 59973 59990 59992]
5949
Selecting with ratios [0.09871667 0.11236667 0.0993     0.10218333 0.09736667 0.09035
 0.09863333 0.10441667 0.09751667 0.09915   ]
Class proportions [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


<ipython-input-3-56e845b0a171>:488: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  order_mg_all = np.array(order_mg_all)
<ipython-input-3-56e845b0a171>:489: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster_sizes_all = np.array(cluster_sizes_all)
/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


...

<ipython-input-3-56e845b0a171>:488: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  order_mg_all = np.array(order_mg_all)
<ipython-input-3-56e845b0a171>:489: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster_sizes_all = np.array(cluster_sizes_all)
/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


eval time on training: 2.2383105754852295
run: 0, grd_normw, subset_size: 0.4, epoch: 2, test_acc: 0.9065999984741211, loss: 0.37805235589777886, best_prec1_gb: 0.9065999984741211, not selected %:41.135
Epoch 3/9
not equal_num
0
[    1    21    34 ... 51396 51416 51431]
4878
1
[    3     6     8 ... 51423 51428 51438]
5384
2
[    5    16    25 ... 51427 51429 51435]
5053
3
[    7    10    12 ... 51422 51424 51440]
5272
4
[    2     9    20 ... 51387 51395 51419]
5180
5
[    0    11    35 ... 51412 51437 51441]
4938
6
[   13    18    32 ... 51426 51430 51442]
4900
7
[   15    29    38 ... 51407 51421 51432]
5280
8
[   17    31    41 ... 51411 51433 51439]
5152
9
[    4    19    22 ... 51417 51434 51436]
5406
Selecting with ratios [0.0948234  0.10465953 0.09822522 0.10248236 0.10069397 0.09598974
 0.09525105 0.10263787 0.10014968 0.10508718]
Class proportions [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


<ipython-input-3-56e845b0a171>:488: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  order_mg_all = np.array(order_mg_all)
<ipython-input-3-56e845b0a171>:489: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster_sizes_all = np.array(cluster_sizes_all)
/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


eval time on training: 2.130307674407959
run: 0, grd_normw, subset_size: 0.4, epoch: 3, test_acc: 0.9096999764442444, loss: 0.3607933155107078, best_prec1_gb: 0.9096999764442444, not selected %:36.26833333333334
Epoch 4/9
not equal_num
0
[    1    21    34 ... 48826 48846 48861]
4562
1
[    3     6     8 ... 48853 48858 48868]
4962
2
[    5    16    25 ... 48857 48859 48865]
4813
3
[    7    10    12 ... 48852 48854 48870]
5010
4
[    2     9    20 ... 48817 48825 48849]
4973
5
[    0    11    35 ... 48834 48842 48867]
4789


KeyboardInterrupt: ignored

In [ ]:
data = np.load(f'{folder}_{subset_size}_{grd}_{runs}_fcraig.npz')
for key, value in data.items():
    np.savetxt("/content/f_craig_10_100/" + key + ".txt", value)

In [ ]:
!zip -r /content/f_craig4.zip /content/f_craig_10_100

  adding: content/f_craig_10_100/ (stored 0%)
  adding: content/f_craig_10_100/fgt_score_time.txt (deflated 54%)
  adding: content/f_craig_10_100/gow_time.txt (deflated 53%)
  adding: content/f_craig_10_100/test_acc.txt (deflated 60%)
  adding: content/f_craig_10_100/craig_time.txt (deflated 53%)
  adding: content/f_craig_10_100/sim_time.txt (deflated 55%)
  adding: content/f_craig_10_100/train_time.txt (deflated 57%)
  adding: content/f_craig_10_100/test_loss.txt (deflated 55%)
  adding: content/f_craig_10_100/times_selected.txt (deflated 97%)
  adding: content/f_craig_10_100/epoch_time.txt (deflated 53%)
  adding: content/f_craig_10_100/train_acc.txt (deflated 56%)
  adding: content/f_craig_10_100/pred_time.txt (deflated 54%)
  adding: content/f_craig_10_100/not_selected.txt (deflated 73%)
  adding: content/f_craig_10_100/grd_time.txt (deflated 53%)
  adding: content/f_craig_10_100/train_loss.txt (deflated 55%)


## Basic CRAIG

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
import numpy as np
import time
from keras.regularizers import l2
#import utilities
import sklearn.metrics
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
total_start = time.time()
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)

num_classes, smtk = 10, 0
Y_train_nocat = Y_train
Y_train = np_utils.to_categorical(Y_train, num_classes)
Y_test = np_utils.to_categorical(Y_test, num_classes)

batch_size = 32
# subset, random = False, False  # all
subset, random = True, False  # greedy
# subset, random = True, True  # random
subset_size = .4 if subset else 1.0
epochs = 10
reg = 1e-4
runs = 3
save_subset = False

folder = f'/content/'


model = Sequential()
model.add(Dense(100, input_dim=784, kernel_regularizer=l2(reg)))
model.add(Activation('sigmoid'))
model.add(Dense(10, kernel_regularizer=l2(reg)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='sgd')


train_loss, test_loss = np.zeros((runs, epochs)), np.zeros((runs, epochs))
train_acc, test_acc = np.zeros((runs, epochs)), np.zeros((runs, epochs))
train_time = np.zeros((runs, epochs))
epoch_time, craig_time = np.zeros((runs, epochs)), np.zeros((runs, epochs))
grd_time, sim_time, pred_time, gow_time = np.zeros((runs, epochs)), np.zeros((runs, epochs)), np.zeros((runs, epochs)), np.zeros((runs, epochs))
not_selected = np.zeros((runs, epochs))
times_selected = np.zeros((runs, len(X_train)))
best_acc = 0
print(f'----------- smtk: {smtk} ------------')

if save_subset:
    B = int(subset_size * len(X_train))
    selected_ndx = np.zeros((runs, epochs, B))
    selected_wgt = np.zeros((runs, epochs, B))

for run in range(runs):
    X_subset = X_train
    Y_subset = Y_train
    W_subset = np.ones(len(X_subset))
    ordering_time,similarity_time, pre_time = 0, 0, 0
    loss_vec, acc_vec, time_vec = [], [], []
    for epoch in range(0, epochs):
        start_e = time.time()
        print('Epoch {}/{}'.format(epoch, epochs - 1))
        num_batches = int(np.ceil(X_subset.shape[0] / float(batch_size)))

        for index in range(num_batches):
            X_batch = X_subset[index * batch_size:(index + 1) * batch_size]
            Y_batch = Y_subset[index * batch_size:(index + 1) * batch_size]
            W_batch = W_subset[index * batch_size:(index + 1) * batch_size]

            start = time.time()
            history = model.train_on_batch(X_batch, Y_batch, sample_weight=W_batch)
            train_time[run][epoch] += time.time() - start
        start_c = time.time()
        if subset:
            if random:
                # indices = np.random.randint(0, len(X_train), int(subset_size * len(X_train)))
                indices = np.arange(0, len(X_train))
                np.random.shuffle(indices)
                indices = indices[:int(subset_size * len(X_train))]
                W_subset = np.ones(len(indices))
            else:
                start = time.time()
                _logits = model.predict(X_train)
                pre_time = time.time() - start
                features = _logits - Y_train
                gow_start = time.time()
                indices, W_subset, _, _, ordering_time, similarity_time = get_orders_and_weights(
                    int(subset_size * len(X_train)), features, 'euclidean', smtk, 0, False, Y_train_nocat)
                go_time = time.time() - gow_start

                W_subset = W_subset / np.sum(W_subset) * len(W_subset)  # todo

            if save_subset:
                selected_ndx[run, epoch], selected_wgt[run, epoch] = indices, W_subset

            grd_time[run, epoch], sim_time[run, epoch], pred_time[run, epoch], gow_time[run, epoch] = ordering_time, similarity_time, pre_time, go_time
            times_selected[run][indices] += 1
            not_selected[run, epoch] = np.sum(times_selected[run] == 0) / len(times_selected[run]) * 100
        else:
            pred_time = 0
            indices = np.arange(len(X_train))
            
        craig_time[run][epoch] = time.time() - start_c
        X_subset = X_train[indices, :]
        Y_subset = Y_train[indices]

        start = time.time()
        score = model.evaluate(X_test, Y_test, verbose=1)
        eval_time = time.time()-start

        start = time.time()
        score_loss = model.evaluate(X_train, Y_train, verbose=1)
        print(f'eval time on training: {time.time()-start}')

        test_loss[run][epoch], test_acc[run][epoch] = score[0], score[1]
        train_loss[run][epoch], train_acc[run][epoch] = score_loss[0], score_loss[1]
        best_acc = max(test_acc[run][epoch], best_acc)

        grd = 'random_wor' if random else 'grd_normw'
        print(f'run: {run}, {grd}, subset_size: {subset_size}, epoch: {epoch}, test_acc: {test_acc[run][epoch]}, '
              f'loss: {train_loss[run][epoch]}, best_prec1_gb: {best_acc}, not selected %:{not_selected[run][epoch]}')
        epoch_time[run][epoch] = time.time() - start_e

    if save_subset:
        print(
            f'Saving the results to {folder}_{subset_size}_{grd}_{runs}')

        np.savez(f'{folder}_{subset_size}_{grd}_{runs}_craig',
                 # f'_{grd}_{args.lr_schedule}_start_{args.start_subset}_lag_{args.lag}_subset',
                 train_loss=train_loss, test_acc=test_acc, train_acc=train_acc, test_loss=test_loss,
                 train_time=train_time, grd_time=grd_time, sim_time=sim_time, pred_time=pred_time,
                 not_selected=not_selected, times_selected=times_selected,
                 subset=selected_ndx, weights=selected_wgt,
                 gow_time = gow_time,
                 epoch_time=epoch_time, craig_time=craig_time)
    else:
        print(
            f'Saving the results to {folder}_{subset_size}_{grd}_{runs}')

        np.savez(f'{folder}_{subset_size}_{grd}_{runs}_craig',
                 # f'_{grd}_{args.lr_schedule}_start_{args.start_subset}_lag_{args.lag}',
                 train_loss=train_loss, test_acc=test_acc, train_acc=train_acc, test_loss=test_loss,
                 train_time=train_time, grd_time=grd_time, sim_time=sim_time, pred_time=pred_time,
                 not_selected=not_selected, times_selected=times_selected,
                 gow_time = gow_time,
                 epoch_time=epoch_time, craig_time=craig_time)
total_time = time.time() - total_start
print(total_time)

In [ ]:
train_loss2,test_acc2,train_acc2,test_loss2,train_time2,grd_time2,sim_time2,pred_time2,gow_time2, total_time2, epoch_time2,craig_time2=train_loss,test_acc,train_acc,test_loss,train_time,grd_time,sim_time,pred_time, gow_time, total_time, epoch_time,craig_time

In [ ]:
data = np.load(f'{folder}_{subset_size}_{grd}_{runs}_craig.npz')
for key, value in data.items():
    np.savetxt("/content/craig1/" + key + ".txt", value)

In [ ]:
!zip -r /content/craig1.zip /content/craig1